In [3]:
import numpy as np
import pandas as pd

In [ ]:
import nltk
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
df = pd.read_csv("../input/the-holy-quran/English.csv", header = None, encoding = "ISO-8859-1")
#Data Snapshot
df.head()

In [ ]:
#Create Surah & Ayat Index
df1 = df[0].str.split("|", expand = True)
df1.head()

In [ ]:
#Connect Surah & Ayat to orginal
df2 = pd.concat([df1,df.iloc[:,1:]], axis = 1)
df2.head()

In [ ]:
#Combine Columns 2:16 in one
df_a = pd.DataFrame()
for a,b in df2.iterrows():
    load = b[2:]    
    data = ''
    for aa in load:
        if str(aa) != 'nan':
            data += str(aa)
    df_a = df_a.append([data])
    
df_a

In [ ]:
df3 = pd.concat([df2.iloc[:,:2],df_a.reset_index().rename(columns = {0:"Data"})], axis = 1).drop(columns = 'index').rename(columns = {0:"Surah",1:'Ayat'})

df3

In [ ]:
surahs = df3['Surah'].unique().tolist()

Surah_Data = []
for surah in surahs:
    Data = ''
    for val in df3[df3["Surah"] == surah]['Data']:
        Data += val
    Surah_Data.append(Data)

In [ ]:
Surahs_df = pd.DataFrame({'Data':Surah_Data}).reset_index().rename(columns = {'index':"Surah"})
Surahs_df

In [ ]:
Surahs_df["Data"]=Surahs_df["Data"].str.replace(';',' ')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('.',' ')
Surahs_df["Data"]=Surahs_df["Data"].str.replace(':','')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('(','')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('"','')
Surahs_df["Data"]=Surahs_df["Data"].str.replace(')','')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('-','')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('Qur´an','Quran')
Surahs_df["Data"]=Surahs_df["Data"].str.replace('Allah´s',"Allah's")
Surahs_df["Data"]=Surahs_df["Data"].str.replace('Isma´il',"Ismail")
Surahs_df["Data"]=Surahs_df["Data"].str.replace('Shu´aib',"Shuaib")
Surahs_df["Data"]=Surahs_df["Data"].str.replace('´partners´',"partners")

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
#Create Stop Words Corpus
stop_words = nltk.corpus.stopwords.words("english") + list(string.punctuation) \
+ list(string.ascii_lowercase) + list(string.ascii_uppercase) + list(string.digits) \
+ list(['--']+["''"]+["``"]+[".."]+["..."]+["ii"]+["iii"]+["iv"]+["'s"]+["the"]+["however"] \
+ ["when"]+["as"]+["meanwhile"]+['eventually'])

In [ ]:
Surah = 1
Data = ''
for a in Surahs_df['Data']:
    Data += a
tokens = nltk.word_tokenize(Data)

# Convert the tokens into lowercase: lower_tokens
lower_tokens = [s.lower() for s in tokens]

filtered_sentence  = [w for w in lower_tokens if not w in stop_words]

#Count Words
hr1_counter = Counter(filtered_sentence)
#Sort Words by counts
sorted_word_counts = sorted(list(hr1_counter.values()), reverse=True)
print(hr1_counter.most_common(50))

In [ ]:
Data_1 = Data.split(' ')

bad_words = {}
bad_words[Data_1[0]] = 1

count = 0
for a in tokens:
    if a.isalpha() == False:
        if a in bad_words.keys():
            bad_words[a] += 1
        else:
            bad_words[a] = 1
        count+=1
bad_df = pd.DataFrame.from_dict(
    bad_words,orient='index').reset_index()
bad_df.sort_values(0,ascending = False).head(30)

In [ ]:
df_vocab = pd.DataFrame.from_dict(hr1_counter,orient = 'index').reset_index().rename(columns = {'index':'Vocab',0:'Count'})
df_vocab.sort_values('Count',ascending = False).head(15)

In [ ]:
Surah = 1
Data = ''
for a in Surahs_df['Data']:
    Data += a
tokens = nltk.word_tokenize(Data)

# Convert the tokens into lowercase: lower_tokens
lower_tokens = [s.lower() for s in tokens]

hr1_filter = [w for w in lower_tokens if not w in stop_words]
#Count Words
hr1_counter = Counter(hr1_filter)
#Sort Words by counts
sorted_word_counts = sorted(list(hr1_counter.values()), reverse=True)
print(hr1_counter.most_common(50))
    
#Set Width & Height
width = 12
height = 12
#Initialize Plot
plt.figure(figsize=(width, height))
#Generate Wordplot
wordcloud = WordCloud(width=1800,height=1400).generate(str(hr1_filter))
plt.imshow(wordcloud)
plt.axis("off")
#Generate Plot
plt.show()